In [1]:
from moonbox import *

In [5]:
ods = [parse_oneday(get_oneday(date=f"2024-03-{x}")) for x in range(1, 32)]
ods

[{'Upper Transit': '04:17',
  'Set': '09:19',
  'phase': 'Waning Gibbous',
  'illumination': 69},
 {'Rise': '00:13',
  'Upper Transit': '05:05',
  'Set': '09:50',
  'phase': 'Waning Gibbous',
  'illumination': 60},
 {'Rise': '01:20',
  'Upper Transit': '05:57',
  'Set': '10:30',
  'phase': 'Waning Crescent',
  'illumination': 49},
 {'Rise': '02:25',
  'Upper Transit': '06:54',
  'Set': '11:20',
  'phase': 'Waning Crescent',
  'illumination': 39},
 {'Rise': '03:27',
  'Upper Transit': '07:53',
  'Set': '12:21',
  'phase': 'Waning Crescent',
  'illumination': 29},
 {'Rise': '04:21',
  'Upper Transit': '08:55',
  'Set': '13:33',
  'phase': 'Waning Crescent',
  'illumination': 19},
 {'Rise': '05:07',
  'Upper Transit': '09:55',
  'Set': '14:50',
  'phase': 'Waning Crescent',
  'illumination': 11},
 {'Rise': '05:45',
  'Upper Transit': '10:53',
  'Set': '16:10',
  'phase': 'Waning Crescent',
  'illumination': 4},
 {'Rise': '06:17',
  'Upper Transit': '11:47',
  'Set': '17:29',
  'phase': 'W

In [10]:
# draw 3 pieces, by quarter:
# draw background
#  - if full, white & done
#  - if new, black & done
#  - if waxing, white
#  - if waning, black
# draw foreground half
#  - if waxing, black 
#  - if waning, white
# draw ellipse

[0,
 1,
 3,
 4,
 8,
 11,
 16,
 19,
 25,
 29,
 35,
 39,
 45,
 49,
 56,
 60,
 65,
 65,
 69,
 74,
 75,
 82,
 83,
 89,
 90,
 94,
 95,
 98,
 98,
 100,
 100]